In [37]:
 
import pandas as pd
import json
from dateutil import parser

In [4]:
ACCOUNT_ID = "101-001-26993136-002"
API_KEY = "a5bc1f4d955885aa1c86dbbc75faa04b-dca8b28605f0e80274551009b7fe445e"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [5]:
session = requests.Session()

In [6]:
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
})

In [ ]:
params = dict(
    count = 10,
    granularity = "H1",
    price = "MBA"
)

In [7]:
params = dict(
        count = 10,
        granularity = 'H1',
        price = "MBA"
    )

In [8]:
url = f"{OANDA_URL}/instruments/EUR_USD/candles"
    

In [11]:
response = session.get(url, params=params, data=None, headers=None)

In [12]:
response.json()

{'instrument': 'EUR_USD',
 'granularity': 'H1',
 'candles': [{'complete': True,
   'volume': 3507,
   'time': '2023-10-20T11:00:00.000000000Z',
   'bid': {'o': '1.05845', 'h': '1.05983', 'l': '1.05842', 'c': '1.05903'},
   'mid': {'o': '1.05852', 'h': '1.05990', 'l': '1.05850', 'c': '1.05910'},
   'ask': {'o': '1.05860', 'h': '1.05997', 'l': '1.05858', 'c': '1.05917'}},
  {'complete': True,
   'volume': 6914,
   'time': '2023-10-20T12:00:00.000000000Z',
   'bid': {'o': '1.05899', 'h': '1.05916', 'l': '1.05644', 'c': '1.05831'},
   'mid': {'o': '1.05906', 'h': '1.05924', 'l': '1.05652', 'c': '1.05838'},
   'ask': {'o': '1.05913', 'h': '1.05932', 'l': '1.05660', 'c': '1.05846'}},
  {'complete': True,
   'volume': 8132,
   'time': '2023-10-20T13:00:00.000000000Z',
   'bid': {'o': '1.05831', 'h': '1.05943', 'l': '1.05783', 'c': '1.05784'},
   'mid': {'o': '1.05838', 'h': '1.05950', 'l': '1.05790', 'c': '1.05791'},
   'ask': {'o': '1.05845', 'h': '1.05957', 'l': '1.05797', 'c': '1.05798'}},

In [13]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [14]:
response = session.get(url, params=None, data=None, headers=None)

In [15]:
response.status_code

200

In [18]:
data = response.json()

In [19]:
instruments_list = data['instruments']

In [20]:
len(instruments_list)

68

In [25]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [26]:
key_i = ['name', 'type', 'displayName', 'pipLocation',
         'displayPrecision', 'tradeUnitsPrecision', 'marginRate']

In [27]:
instruments_dict = {}
for i in instruments_list:
    key = i['name']
    instruments_dict[key] = { k: i[k] for k in key_i }

In [29]:
instruments_dict['USD_CAD']

{'name': 'USD_CAD',
 'type': 'CURRENCY',
 'displayName': 'USD/CAD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.02'}

In [30]:
# -4 -> 0.0001
pow(10, -4)

0.0001

In [31]:
with open("../data/instruments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [43]:
def fetch_candles(pair_name, count=10, granularity="H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()
    
    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    return response.status_code, data

def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame.empty
    
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']
    
    final_data = []
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']
        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"] = float(candle[p][o])
        final_data.append(new_dict)
    df = pd.DataFrame.from_dict(final_data)
    return df

def create_data_file(pair_name, count=10, granularity="H1"):
    code, data = fetch_candles(pair_name, count, granularity)
    if code != 200:
        print("Failed", pair_name, data)
        return
    if len(data) == 0:
        print("No candles", pair_name)
    candles_df = get_candles_df(data)
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")
    print(f"{pair_name} {granularity} {candles_df.shape[0]} candles, {candles_df.time.min()} {candles_df.time.max()}")
    

In [39]:
code, data = fetch_candles("EUR_USD", count=10, granularity="H4")
candles_df = get_candles_df(data)

In [40]:
candles_df

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2023-10-19 05:00:00+00:00,16874,1.05335,1.05482,1.05282,1.05359,1.05328,1.05474,1.05274,1.05352,1.05342,1.05490,1.05289,1.05366
1,2023-10-19 09:00:00+00:00,18582,1.05361,1.05797,1.05352,1.05763,1.05354,1.05790,1.05345,1.05756,1.05368,1.05804,1.05360,1.05770
2,2023-10-19 13:00:00+00:00,39637,1.05767,1.06168,1.05564,1.05966,1.05760,1.06160,1.05556,1.05957,1.05774,1.06175,1.05573,1.05974
3,2023-10-19 17:00:00+00:00,23883,1.05964,1.06066,1.05783,1.05814,1.05956,1.06059,1.05776,1.05804,1.05972,1.06074,1.05790,1.05824
4,2023-10-19 21:00:00+00:00,5602,1.05838,1.05868,1.05738,1.05760,1.05792,1.05861,1.05730,1.05753,1.05885,1.05885,1.05745,1.05768
5,2023-10-20 01:00:00+00:00,8879,1.05760,1.05800,1.05684,1.05754,1.05752,1.05793,1.05677,1.05746,1.05768,1.05808,1.05691,1.05761
6,2023-10-20 05:00:00+00:00,16930,1.05750,1.05953,1.05650,1.05877,1.05743,1.05946,1.05643,1.05869,1.05758,1.05960,1.05658,1.05885
7,2023-10-20 09:00:00+00:00,17538,1.05875,1.05990,1.05652,1.05838,1.05868,1.05983,1.05644,1.05831,1.05882,1.05997,1.05660,1.05846
8,2023-10-20 13:00:00+00:00,24510,1.05838,1.06009,1.05722,1.05896,1.05831,1.06002,1.05714,1.05888,1.05845,1.06016,1.05730,1.05903
9,2023-10-20 17:00:00+00:00,10864,1.05898,1.06038,1.05864,1.05948,1.05891,1.06031,1.05857,1.05938,1.05905,1.06045,1.05871,1.05957


In [41]:
candles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype                  
---  ------  --------------  -----                  
 0   time    10 non-null     datetime64[ns, tzutc()]
 1   volume  10 non-null     int64                  
 2   mid_o   10 non-null     float64                
 3   mid_h   10 non-null     float64                
 4   mid_l   10 non-null     float64                
 5   mid_c   10 non-null     float64                
 6   bid_o   10 non-null     float64                
 7   bid_h   10 non-null     float64                
 8   bid_l   10 non-null     float64                
 9   bid_c   10 non-null     float64                
 10  ask_o   10 non-null     float64                
 11  ask_h   10 non-null     float64                
 12  ask_l   10 non-null     float64                
 13  ask_c   10 non-null     float64                
dtypes: datetime64[ns, tzutc()](1), float64(12), i

In [44]:
create_data_file("EUR_USD", count=10, granularity="H4")

EUR_USD H4 10 candles, 2023-10-19 05:00:00+00:00 2023-10-20 17:00:00+00:00


In [45]:
our_curr = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD', 'AUD']

In [46]:
for p1 in our_curr:
    for p2 in our_curr:
        pr = f"{p1}_{p2}"
        if pr in instruments_dict:
            for g in ["H1", "H4"]:
                create_data_file(pr, count=4001, granularity=g)

EUR_USD H1 4001 candles, 2023-03-02 05:00:00+00:00 2023-10-20 20:00:00+00:00
EUR_USD H4 4001 candles, 2021-03-31 01:00:00+00:00 2023-10-20 17:00:00+00:00
EUR_GBP H1 4001 candles, 2023-03-02 05:00:00+00:00 2023-10-20 20:00:00+00:00
EUR_GBP H4 4001 candles, 2021-03-31 01:00:00+00:00 2023-10-20 17:00:00+00:00
EUR_JPY H1 4001 candles, 2023-03-02 05:00:00+00:00 2023-10-20 20:00:00+00:00
EUR_JPY H4 4001 candles, 2021-03-30 17:00:00+00:00 2023-10-20 17:00:00+00:00
EUR_CHF H1 4001 candles, 2023-03-02 05:00:00+00:00 2023-10-20 20:00:00+00:00
EUR_CHF H4 4001 candles, 2021-03-31 01:00:00+00:00 2023-10-20 17:00:00+00:00
EUR_NZD H1 4001 candles, 2023-03-02 04:00:00+00:00 2023-10-20 20:00:00+00:00
EUR_NZD H4 4001 candles, 2021-03-31 01:00:00+00:00 2023-10-20 17:00:00+00:00
EUR_CAD H1 4001 candles, 2023-03-02 05:00:00+00:00 2023-10-20 20:00:00+00:00
EUR_CAD H4 4001 candles, 2021-03-31 01:00:00+00:00 2023-10-20 17:00:00+00:00
EUR_AUD H1 4001 candles, 2023-03-02 04:00:00+00:00 2023-10-20 20:00:00+00:00